<a href="https://colab.research.google.com/github/asiat2/BigDataSQL-Python/blob/main/BigData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Install PySpark
!pip install pyspark
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3


### **Loading the dataset**

In [ ]:
df1 = pd.read_csv('/content/drive/MyDrive/ColabNotebooks/dataset/name.basics.csv')
df2 = pd.read_csv('/content/drive/MyDrive/ColabNotebooks/dataset/title.akas.csv')
df3 = pd.read_csv('/content/drive/MyDrive/ColabNotebooks/dataset/title.basics.csv')
df4 = pd.read_csv('/content/drive/MyDrive/ColabNotebooks/dataset/title.principals.csv')
df5 = pd.read_csv('/content/drive/MyDrive/ColabNotebooks/dataset/title.ratings.csv')

In [ ]:
print(df1.head())
print(df2.head())
print(df3.head())
print(df4.head())
print(df5.head())

### **Data Transformation**

In [ ]:
df1.columns = df1.iloc[0]  # Set the first row as the header
df1 = df1[1:].reset_index(drop=True)  # Drop the first row and reset the index

In [ ]:
# Rename 'titleId' to 'tconst'
df2.rename(columns={"titleId": "tconst"}, inplace=True)

In [ ]:
#checking data type and column names
print(df1.info())
print(df2.info())
print(df3.info())
print(df4.info())
print(df5.info())

In [ ]:
# Create an SQLite database in memory
conn = sqlite3.connect(":memory:")

In [ ]:
# Load DataFrames into SQLite tables
df1.to_sql('/content/drive/MyDrive/ColabNotebooks/dataset/name.basics.csv', conn, index=False, if_exists='replace')
df2.to_sql('/content/drive/MyDrive/ColabNotebooks/dataset/title.akas.csv', conn, index=False, if_exists='replace')
df3.to_sql('/content/drive/MyDrive/ColabNotebooks/dataset/title.basics.csv', conn, index=False, if_exists='replace')
df4.to_sql('/content/drive/MyDrive/ColabNotebooks/dataset/title.principals.csv', conn, index=False, if_exists='replace')
df5.to_sql('/content/drive/MyDrive/ColabNotebooks/dataset/title.ratings.csv', conn, index=False, if_exists='replace')

### **Write an SQL query to merge the datasets**




In [ ]:
# Extract relevant columns from each dataset
df1_filtered = df1[['nconst']]  # Only needed for linking principal roles
df2_filtered = df2[['tconst', 'region']]
df3_filtered = df3[['tconst', 'genres', 'runtimeMinutes','startYear']]
df4_filtered = df4[['tconst', 'nconst', 'category']]  # Includes nconst for linking
df5_filtered = df5[['tconst', 'averageRating', 'numVotes']]

# Merge datasets
merged_data = (
    df3_filtered
    .merge(df2_filtered, on='tconst', how='inner')  # Merge with title_akas
    .merge(df5_filtered, on='tconst', how='inner')  # Merge with title_ratings
    .merge(df4_filtered, on='tconst', how='inner')  # Merge with title_principals
    .merge(df1_filtered, on='nconst', how='inner')  # Merge with name_basics
)

# Display the merged data
print(merged_data.head())


In [ ]:
# Save the merged data to a CSV file
merged_data.to_csv('merged_data.csv', index=False)

# Print a confirmation message
print("Merged dataset saved to 'merged_data.csv'.")


###**Loading the merged dataset**

In [ ]:
sample = pd.read_csv('merged_data.csv')
sample.head()

In [ ]:
# to read last five rows
sample.tail()

In [ ]:
sample.info()

In [ ]:
# to know the rows and column of the data
sample.shape

In [ ]:
duplicate_rows = sample[sample.duplicated()]
print("Number of duplicate rows:", duplicate_rows.shape[0])

In [ ]:
#to removed duplicate
sample.drop_duplicates(inplace = True)

In [ ]:
duplicate_rows = sample[sample.duplicated()]
print("Number of duplicate rows:", duplicate_rows.shape[0])

In [ ]:
count_backslash_N = sample.isin(['\\N']).sum()
print(count_backslash_N)

In [ ]:
# Filter out rows where any column contains the value '\N' (often used as a placeholder for missing or invalid data).
# This ensures the DataFrame only retains rows without '\N' in any column.
sample = sample[~sample.isin(['\\N']).any(axis=1)]
count_backslash_N = sample.isin(['\\N']).sum()
print(count_backslash_N)

In [ ]:
sample.shape

In [ ]:
#statistical summary
sample.describe()

In [ ]:
# Save the sample data to a CSV file
sample.to_csv('sample.csv', index=False)

# Print a confirmation message
print("sample dataset saved to 'sample.csv'.")


###**The insight of te Analysis**

In [ ]:
# Group by region and genres to count occurrences
popular_genres = (
    sample.groupby(['region', 'genres'])
    .size()
    .reset_index(name='genre_count')
    .sort_values(by='genre_count', ascending=False)
)

# Display top results
print(popular_genres.head(10))

# Visualization
import seaborn as sns
import matplotlib.pyplot as plt

sns.barplot(data=popular_genres.head(10), x='genre_count', y='genres', hue='region')
plt.title('Most Popular Genres by Region')
plt.show()


In [ ]:
# Check correlation between runtime and average ratings
runtime_ratings = sample[['runtimeMinutes', 'averageRating']].dropna()
correlation = runtime_ratings.corr()
print(f"Correlation between Runtime and Average Ratings: \n{correlation}")

# Scatter plot with color based on averageRating
plt.figure(figsize=(10, 6))
scatter = plt.scatter(
    runtime_ratings['runtimeMinutes'],
    runtime_ratings['averageRating'],
    c=runtime_ratings['averageRating'],  # Use averageRating for color
    cmap='viridis',  # Color map
    alpha=0.8  # Transparency
)
plt.colorbar(scatter, label='Average Rating')  # Add a color bar
plt.title('Runtime vs. Average Ratings (Colored by Average Rating)')
plt.xlabel('Runtime (Minutes)')
plt.ylabel('Average Rating')

plt.show()


In [ ]:
# Group by region and calculate the average rating
avg_ratings_by_region = (
    sample.groupby('region')['averageRating']
    .mean()
    .reset_index()
    .sort_values(by='averageRating', ascending=False)
)

# Display top regions
print(avg_ratings_by_region.head())

# Visualization
sns.barplot(data=avg_ratings_by_region.head(10), x='averageRating', y='region')
plt.title('Average Rating by Region')
plt.show()


In [ ]:
# Analyze the relationship between votes and ratings
votes_ratings = sample[['numVotes', 'averageRating']].dropna()

# Scatter plot with color based on numVotes
plt.figure(figsize=(10, 6))
scatter = plt.scatter(
    votes_ratings['numVotes'],
    votes_ratings['averageRating'],
    c=votes_ratings['numVotes'],  # Use numVotes for color
    cmap='viridis',  # Color map
    alpha=0.8  # Transparency for better visibility
)
plt.colorbar(scatter, label='Number of Votes (Log Scale)')  # Add a color bar
plt.title('Votes vs. Ratings (Colored by Number of Votes)')
plt.xscale('log')  # Log scale for better visualization
plt.xlabel('Number of Votes (Log Scale)')
plt.ylabel('Average Rating')
plt.show()

# Correlation
correlation = votes_ratings.corr()
print(f"Correlation between Votes and Ratings: \n{correlation}")



In [ ]:
# Group by category (principal roles) and calculate average rating
ratings_by_role = (
    sample.groupby('category')['averageRating']
    .mean()
    .reset_index()
    .sort_values(by='averageRating', ascending=False)
)

# Display results
print(ratings_by_role)

# Visualization
sns.barplot(data=ratings_by_role, x='averageRating', y='category')
plt.title('Average Rating by Principal Role')
plt.show()
